### Y_Quality 처음부터 drop하고 train


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pycaret

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.2/320.2 KB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 KB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.7/147.7 KB 310.2 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 KB 13.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 662.4/662.4 KB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.8/25.8 MB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [3]:
!pip install numpy==1.21

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 59.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.3.post1 requires numpy<1.20,>=1.16.0, but you have numpy 1.21.0 which is incompatible.
xarray-einstats 0.5.1 requires scipy>=1.6, but you have scipy 1.5.4 which is incompatible.
numba 0.54.1 requires numpy<1.21,>=1.17, but you have numpy 1.21.0 which is incompatible.
en-core-web-sm 3.4.1 requires spacy<3.5.0,>=3.4.0, but you have spacy 2.3.9 which is incompatible.


In [5]:
import pandas as pd
import random
import os
import numpy as np

from pycaret.classification import *

In [6]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(37) # Seed 고정

In [7]:
re_x = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/LG Aimers Dacon/re_x_timestamp.csv")

features = list(re_x.columns)
features.remove('Unnamed: 0')

In [8]:
train_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/LG Aimers Dacon/open/train.csv")
test_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/LG Aimers Dacon/open/test.csv")

In [16]:
# data에서 모든 값이 결측치인 X 삭제

train = train_df.copy()

null = train.isnull().sum()
null = null.to_frame().transpose()
Xs = null.columns[6:]
Nan_list = []

for x in Xs:
  if float(null[x]) == len(train):
    Nan_list.append(x)

train.drop(Nan_list, axis = 1, inplace = True)

test = test_df.copy()
test.drop(Nan_list, axis = 1, inplace = True)


# RF2에서 얻은 feature들만 남기기
to_remove = list(set(list(train.columns)[4:]) - set(features))
train.drop(to_remove, axis = 1, inplace = True)
test.drop(to_remove, axis = 1, inplace = True)

train.drop(["Y_Quality"], axis = 1, inplace = True)

In [18]:
# 1. 데이터 준비
setup_clf = setup(data=train, target='Y_Class', train_size=0.7, 
                  normalize=True, normalize_method='minmax',
                  session_id=777)

,Description,Value
0,session_id,777
1,Target,Y_Class
2,Target Type,Multiclass
3,Label Encoded,None
4,Original Data,"(598, 1608)"
5,Missing Values,True
6,Numeric Features,1301
7,Categorical Features,304
8,Ordinal Features,False
9,High Cardinality Features,False


INFO:logs:create_model_container: 0
INFO:logs:master_model_container: 0
INFO:logs:display_container: 1
INFO:logs:Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=True, features_todrop=[],
                                      id_columns=[],
                                      ml_usecase='classification',
                                      numerical_features=[], target='Y_Class',
                                      time_features=[])),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numeric_stra...
                ('P_transform', 'passthrough'), ('binn', 'passthrough'),
                ('rem_outliers', 'passthrough'), ('cluster_all', 'passthrough'),
                ('du

In [19]:
# 2. 모델 비교
model = compare_models(sort='Accuracy', fold = 5, n_select=5)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.7728,0.7569,0.5914,0.7630,0.7526,0.4462,0.4662,1.178
gbc,Gradient Boosting Classifier,0.7680,0.7812,0.5967,0.7672,0.7511,0.4527,0.4709,7.672
rf,Random Forest Classifier,0.7656,0.7717,0.5581,0.7625,0.7387,0.4122,0.4454,0.434
et,Extra Trees Classifier,0.7584,0.7605,0.5345,0.7555,0.7241,0.3751,0.4152,0.394
lda,Linear Discriminant Analysis,0.7536,0.7383,0.5462,0.7465,0.7252,0.3792,0.4115,0.556
knn,K Neighbors Classifier,0.7442,0.7225,0.5257,0.7378,0.7130,0.3462,0.3731,0.298
lr,Logistic Regression,0.7441,0.7314,0.5241,0.7340,0.7113,0.3470,0.3789,1.006
svm,SVM - Linear Kernel,0.7225,0.0000,0.5075,0.6938,0.6893,0.3127,0.3339,0.106
nb,Naive Bayes,0.7178,0.6087,0.4713,0.6845,0.6730,0.2559,0.2788,0.082
ridge,Ridge Classifier,0.7106,0.0000,0.5420,0.6966,0.6942,0.3280,0.3373,0.074


INFO:logs:create_model_container: 14
INFO:logs:master_model_container: 14
INFO:logs:display_container: 2
INFO:logs:[LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=777, reg_alpha=0.0, reg_lambda=0.0, silent='warn',
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0), GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.

In [20]:
# 3. 모델 튜닝 및 앙상블
# 모델 튜닝
tuned_model = [tune_model(i) for i in model]

# 모델 앙상블
blended_model = blend_models(estimator_list=tuned_model)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6905,0.6200,0.4215,0.5858,0.6200,0.1429,0.1785
1,0.8571,0.8076,0.7707,0.8546,0.8535,0.6822,0.6861
2,0.8333,0.8805,0.6349,0.8657,0.8088,0.5676,0.6300
3,0.7857,0.7032,0.5679,0.7751,0.7518,0.4441,0.4929
4,0.6905,0.7539,0.4937,0.6704,0.6742,0.2927,0.2976
5,0.7143,0.6225,0.4973,0.6756,0.6850,0.3048,0.3187
6,0.7381,0.7304,0.5322,0.7583,0.7140,0.3740,0.3870
7,0.8333,0.9329,0.6885,0.8484,0.8201,0.5950,0.6233
8,0.7317,0.7552,0.5084,0.7096,0.7074,0.3083,0.3248


INFO:logs:create_model_container: 20
INFO:logs:master_model_container: 20
INFO:logs:display_container: 8
INFO:logs:VotingClassifier(estimators=[('lightgbm',
                              LGBMClassifier(bagging_fraction=0.8,
                                             bagging_freq=4,
                                             boosting_type='gbdt',
                                             class_weight=None,
                                             colsample_bytree=1.0,
                                             feature_fraction=0.9,
                                             importance_type='split',
                                             learning_rate=0.05, max_depth=-1,
                                             min_child_samples=26,
                                             min_child_weight=0.001,
                                             min_split_gain=0.3,
                                             n_estimators=230, n_jobs=-1,
                          

In [21]:
# 모델 성능평가
final_model = finalize_model(blended_model)
evaluate_model(final_model)

INFO:logs:Initializing finalize_model()
INFO:logs:finalize_model(estimator=VotingClassifier(estimators=[('lightgbm',
                              LGBMClassifier(bagging_fraction=0.8,
                                             bagging_freq=4,
                                             boosting_type='gbdt',
                                             class_weight=None,
                                             colsample_bytree=1.0,
                                             feature_fraction=0.9,
                                             importance_type='split',
                                             learning_rate=0.05, max_depth=-1,
                                             min_child_samples=26,
                                             min_child_weight=0.001,
                                             min_split_gain=0.3,
                                             n_estimators=230, n_jobs=-1,
                                             num_leaves=70, object

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [22]:
from pycaret.utils import check_metric

prediction = predict_model(final_model, data=test)
# print("테스트셋 정확도: "+str(check_metric(prediction['Label'], test['diagnosis'], metric = 'Accuracy')))

INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=VotingClassifier(estimators=[('lightgbm',
                              LGBMClassifier(bagging_fraction=0.8,
                                             bagging_freq=4,
                                             boosting_type='gbdt',
                                             class_weight=None,
                                             colsample_bytree=1.0,
                                             feature_fraction=0.9,
                                             importance_type='split',
                                             learning_rate=0.05, max_depth=-1,
                                             min_child_samples=26,
                                             min_child_weight=0.001,
                                             min_split_gain=0.3,
                                             n_estimators=230, n_jobs=-1,
                                             num_leaves=70, objectiv

In [36]:
pd.DataFrame(prediction["Label"]).describe()

,Label
count,310.000000
mean,0.912903
std,0.334859
min,0.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,2.000000


In [31]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310 entries, 0 to 309
Columns: 1607 entries, PRODUCT_ID to X_2871
dtypes: float64(1603), object(4)
memory usage: 3.8+ MB


In [32]:
submit = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/LG Aimers Dacon/open/sample_submission.csv")

submit['Y_Class'] = prediction["Label"]

submit.to_csv('AutoML_submission.csv', index = False)